# Import

## import packages

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

from modules.layers import hamming_layer, maxnet_layer, kohonen_layer
from modules.HammingNN import Hamming_NN
from modules.KohonenSOM import KohonenSOM

## import data

In [ ]:
# Forgor how to choose dimensions for reshape, but I will change 
# do something about this next time I'll use this code... No promise tho 
def show(array, NM, title = False, size = (10, 5)):
    n, m = NM
    fig, axs = plt.subplots(n, m, figsize=size)
    array_to_plot = array.reshape((n, m) + array.shape[1:])
    for i in range(n):
        for j in range(m):
            axs[i,j].imshow(array_to_plot[i,j], cmap='Greys')
            if title:
                axs[i,j].title.set_text((i*m) + j)

In [ ]:
def load_data(num_type, threshold = 0.5):
    if num_type == 'digits':
        dim = 8
        digits = load_digits()
        X = digits.data / digits.data.max(axis = 1, keepdims = True) >= threshold
        Y = digits.target

    elif num_type == 'mnist':
        dim = 28
        data = pd.read_csv("./data/train.csv").to_numpy()
        X = data[:, 1:] / 255. >= threshold
        Y = data[:, 0]
    return X, Y, dim

X, Y, dim = load_data('mnist', 0.5)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=101)

# Creating a model

In [ ]:
SOM = KohonenSOM(kohonen_layer(dim * dim, 1))

SOM.train(X_train, epochs=10, lr=0.9, radius=5, lr_decay=0.4, r_decay=0.25, R_0 = 12.5)

In [ ]:
SOM.layer.weights.shape

In [ ]:
show(SOM.layer.weights.reshape(-1, dim, dim), NM = (2, 10), size = (20,10), title=True)

In [ ]:
E = SOM.layer.weights[[16, 19, 17, 11, 9, 13, 3, 8, 7, 14]] >= 0.4
show(E.reshape(-1, dim, dim), NM = (2, 5), size = (20,10))

In [ ]:
# E = SOM.layer.weights

M, N = E.shape
        
# activation functions:
# ls, relu, sgelu, ssilu, smish
layers = [
    hamming_layer(N, M, activation="ls"),
    maxnet_layer(M, M, activation="ls"),
]

model = Hamming_NN(layers, E)

predictions = model.predict_class_separated(X_test, y_test, 100)

In [ ]:
print("Size of the test array: {}\nSum of false predictions: {}".format(X_test.shape[0], np.sum(predictions[1])))

In [ ]:
data = [
    go.Bar(x = np.arange(10), y = (1 - predictions[0])*100),
]

layout = go.Layout(
    height = 500,
    title = dict(
        text = 'Error on each mnist number',
        font_size = 30,
        x = .5
    ),
    xaxis = dict(nticks = 11),
    xaxis_title = dict(
        text = 'Number',
        font_size = 20
    ),
    yaxis=dict(ticksuffix="%"),
    yaxis_title = dict(
        text = 'Percentage of error, [%]',
        font_size = 20
    ),
)
go.Figure(data, layout)